# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [74]:
weather_file = "../WeatherPy/weather_data.csv"
weather_file_df = pd.read_csv(weather_file)
weather_file_df.head()

,Name,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,New Norfolk,-42.7826,147.0587,53.58,89,100,1.34,AU,1623718407
1,Rikitea,-23.1203,-134.9692,75.06,81,100,12.41,PF,1623718408
2,Port Hawkesbury,45.6169,-61.3485,53.73,95,90,5.99,CA,1623718409
3,Alyangula,-13.8483,136.4192,80.47,83,100,4.61,AU,1623718410
4,Puerto Ayora,-0.7393,-90.3518,73.54,86,100,3.00,EC,1623718410


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [19]:
gmaps.configure(api_key=g_key)

In [24]:
locations = weather_file_df[["Lat", "Lng"]].astype(float)
weight = weather_file_df["Humidity"].astype(float)

In [46]:
humidity_max = weather_file_df["Humidity"].max().astype(float)
humidity_max

100.0

In [48]:
fig = gmaps.figure(zoom_level=2, center=(-3.8,102))

heat_layer = gmaps.heatmap_layer(locations, weights=weight, dissipating=False, max_intensity=humidity_max, point_radius=2)

fig.add_layer(heat_layer)

fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [73]:
ideal_weather = weather_file_df.loc[((weather_file_df["Max Temp"] > 65) & (weather_file_df["Max Temp"] < 83)) &
                                  ((weather_file_df["Cloudiness"] < 8) & (weather_file_df["Cloudiness"] > 1)) &
                                   (weather_file_df["Wind Speed"] < 15) & (weather_file_df["Humidity"] < 80)]

ideal_weather

,Name,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
23,Ponta do Sol,32.6667,-17.1000,68.63,79,5,4.36,PT,1623718424
44,Gurupi,-11.7292,-49.0686,78.08,44,2,3.47,BR,1623718439
184,Kushmurun,52.4503,64.6311,69.78,42,5,4.76,KZ,1623718540
210,Resistencia,-27.4606,-58.9839,65.05,74,6,6.91,AR,1623718281
306,Hasaki,35.7333,140.8333,75.60,69,2,5.82,JP,1623718629
315,Barcelos,41.5388,-8.6151,71.28,73,2,4.25,PT,1623718635
338,Hamilton,39.1834,-84.5333,79.43,60,7,7.34,US,1623718654
349,Sechura,-5.5569,-80.8222,69.82,75,4,11.30,PE,1623718661
482,Springfield,37.2153,-93.2982,82.44,60,6,6.38,US,1623718672
566,Baruun-Urt,46.6806,113.2792,69.57,43,6,5.75,MN,1623718829


In [82]:
all_indexes = weather_file_df[~((weather_file_df["Max Temp"] > 65) & (weather_file_df["Max Temp"] < 83) &
                                  ((weather_file_df["Cloudiness"] < 8) & (weather_file_df["Cloudiness"] > 1)) &
                                   (weather_file_df["Wind Speed"] < 15) & (weather_file_df["Humidity"] < 80))].index

weather_file_df.drop(all_indexes, inplace = True)

weather_file_df

,Name,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
23,Ponta do Sol,32.6667,-17.1000,68.63,79,5,4.36,PT,1623718424
44,Gurupi,-11.7292,-49.0686,78.08,44,2,3.47,BR,1623718439
184,Kushmurun,52.4503,64.6311,69.78,42,5,4.76,KZ,1623718540
210,Resistencia,-27.4606,-58.9839,65.05,74,6,6.91,AR,1623718281
306,Hasaki,35.7333,140.8333,75.60,69,2,5.82,JP,1623718629
315,Barcelos,41.5388,-8.6151,71.28,73,2,4.25,PT,1623718635
338,Hamilton,39.1834,-84.5333,79.43,60,7,7.34,US,1623718654
349,Sechura,-5.5569,-80.8222,69.82,75,4,11.30,PE,1623718661
482,Springfield,37.2153,-93.2982,82.44,60,6,6.38,US,1623718672
566,Baruun-Urt,46.6806,113.2792,69.57,43,6,5.75,MN,1623718829


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
